In [ ]:
# final
from gurobipy import *
from operator import le
from unicodedata import name
from numpy import conjugate
import random
from typing import List
import pandas as pd
from gurobipy import GRB
import numpy as np


In [ ]:
def discreteTime(timeWindow, intervalSize):   # O(lenghtOftimeWindow/size)
    tw = []
    currtime = timeWindow[0]
    while (currtime <= timeWindow[1]):
        tw.append(currtime)
        currtime = currtime+intervalSize
    return tw


def distinctCustomer(path,nodeCount):
    customer = 0
    visited = [] # stores index of visited customer
    for i in range(1,len(path)-1):
        node = path[i]
        for j in range(0,len(nodeCount)):
            if(node in nodeCount[j]):
                if(j not in visited):
                    visited.append(j)
                    customer+=1
    return customer,visited
def getmin(path,ans,new_edges):
    for i in range(0,len(path)-1):
        edge = (path[i],path[i+1])
        minVal = 99 # any number > 1
        for key in new_edges:
            if(new_edges[key] == edge):
                minVal = min(minVal,ans[key])
    return minVal

def maxCustomerpath(paths,nodeCount):
    maxPaths = 0
    maxCustomer = 0
    for i in range(0,len(paths)):
        path = paths[i]
        customer,visited = distinctCustomer(path,nodeCount)
        if(maxCustomer < customer):
            maxCustomer = customer
            maxPaths = visited # customers visited
            pathdirection = path
            
    return maxCustomer,maxPaths,pathdirection

def mycallback(model, where):

    if where == GRB.Callback.BARRIER:
        # Barrier callback
        itcnt = model.cbGet(GRB.Callback.BARRIER_ITRCNT)
        primobj = model.cbGet(GRB.Callback.BARRIER_PRIMOBJ)
        dualobj = model.cbGet(GRB.Callback.BARRIER_DUALOBJ)
        priminf = model.cbGet(GRB.Callback.BARRIER_PRIMINF)
        dualinf = model.cbGet(GRB.Callback.BARRIER_DUALINF)
        cmpl = model.cbGet(GRB.Callback.BARRIER_COMPL)
        if abs(primobj - dualobj) <= 5:
            print("Custom gap reached")
            model.terminate()
        print("%d %g %g %g %g %g" % (itcnt, primobj, dualobj, priminf, dualinf, cmpl))
        
def getCustomer(node,nodeCount):
    for i in range(len(nodeCount)):
        if(node in nodeCount[i]):
            return i
         
    
def length(u,v,nodeCount,path,idx,ts):
    cust_u = getCustomer(u,nodeCount)
    cust_v = getCustomer(v,nodeCount)
    if(cust_u == cust_v):
        return 0
    # x will start from v and go back to see if cust(x) = cust_u
#     route = []
#     route.append(v)
    curr = idx
    while(True):
        if(curr == 0):
            break
        prev = path[curr]
#         route.append(ts[prev])
        if(getCustomer(ts[prev],nodeCount) == cust_u):
            return 0
        curr = prev
    return 1
        
        
    
    
    
# route = []
#     route.append(-1)
#     curr = -1
#     while(True):
#         if(curr == 0):
#             break
#         prev = path[curr]
#         route.append(ts[prev])
#         curr = prev 
    
def dfs(v,visited,stack,graph):
        visited[v] = True
        for i in graph[v]:
            if visited[i] == False:
                dfs(i,visited,stack,graph)
        stack.insert(0,v)
def topologicalSort(visited,graph):
        stack =[] 
        dfs(0,visited,stack,graph)
        print ("topologically sorted",stack)
        return stack

def onesPath(graph: List[List[int]],edges1:List[List[int]]) -> List[List[int]]:
    stack = [[0]]
    s=[[]]
    paths = []
    ed=[]
    while stack:
        path = stack.pop()
        e=s.pop()
        node = path[-1]
        if node == len(graph) - 1:
            paths.append(path)
            ed.append(e)
            continue
            
        for i in range(len(graph[node])):
            if(ans[edges1[node][i]]==1):
                stack.append((path + [graph[node][i]]))
                s.append((e+ [edges1[node][i]]))       
            
    return  paths
def searchKey(customer,cust_dict):
    for key in cust_dict.keys():        
        if customer == cust_dict[key]:
            return key

In [ ]:
customers = 25
file = "R111.csv"
df = pd.read_csv(file) # starting @ 2:07
start = 0
speed = 1 # km/min
df['XCOORD.'] = df['XCOORD.'] - df['XCOORD.'][0]
df['YCOORD.'] = df['YCOORD.'] - df['YCOORD.'][0]
# df = df[ df['DUE DATE'] >= start + (df['XCOORD.']+df['YCOORD.'])/speed ]
# df.reset_index(inplace = True)

serviceTime = df['SERVICE TIME'][1]
#remove unreachable if(starting time + travelling > end )


print(len(df))
# df = df[ df['DUE DATE'] >= start + (df['XCOORD.']+df['YCOORD.'])/speed ]
timeWindows = [] 
for i in range(0,customers):
    st = df['READY TIME'][i+1]
    if(st%10 != 0):
        st=st + (10 - df['READY TIME'][i+1]%10)
    et = df['DUE DATE'][i+1]
    if(et%10 != 0):
        et = et +(10 - df['DUE DATE'][i+1]%10)
    tw = [st,et]
    timeWindows.append(tw)

discreteTimeWindows = []
for i in timeWindows:
    discreteTimeWindows.append(discreteTime(i,10))

time_from_source = []
for i in range(0,customers):
    time_from_source.append(round(math.sqrt((df['XCOORD.'][i+1])**2  + (df['YCOORD.'][i+1])**2 ),2))
    
    

timeMatrix = []

for i in range(0,customers):
    timeMatrix.append([0]*customers)
for i in range(0,customers):
    for j in range(0,customers):
        if(i==j):
            timeMatrix[i][j] = 0
        elif (i<j):
#             c= int(math.sqrt(abs(df['XCOORD.'][i+1]-df['XCOORD.'][j+1]))+abs(df['YCOORD.'][i+1]-df['YCOORD.'][j+1]))
            euclidean_distance= round( math.sqrt((df['XCOORD.'][i+1]-df['XCOORD.'][j+1])**2  + (df['YCOORD.'][i+1]-df['YCOORD.'][j+1])**2 ),2)
            timeMatrix[i][j] = round(euclidean_distance,2)
            timeMatrix[j][i] = round(euclidean_distance,2)




In [ ]:
total_runtime = 0
cust_dict = {}
for i in range(customers):
    cust_dict[i] = i

final_path_list = []
Iterations = 0
    
VEHICLES = 0
customers_covered = 0

while(len(timeWindows)!=0):
    Iterations+=1
    m1 = Model('VRPTW')
    
    count = 0
    nodeCount = []  # assigns a number to node using count and stores its value

    for i in range(0, len(discreteTimeWindows)):
        c = []
        for j in range(0, len(discreteTimeWindows[i])):
            c.append(count+1)
            count += 1
        nodeCount.append(c)

    print("Number of customers",len(nodeCount))
    print("Nodes ",nodeCount)
    print("time from source",time_from_source)



    edgeFlows = {}
    edgeNumber = 0
    x = [] 
    edges = {}
    edgesList = []  # stores edge number going out of ith node
    edgesList.append([])  # for s
    
    
    
    # GRAPH CONSTRUCTION
    # out of s to starting of each customer
    e = []
    for i in range(0, len(nodeCount)):
            reaching_time = time_from_source[i]
            if(reaching_time <= discreteTimeWindows[i][0]):
                x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
                edges['x[%i]' % edgeNumber] = (0, nodeCount[i][0])
                edgeFlows[edgeNumber] = (0, nodeCount[i][0])
                e.append(edgeNumber)
                edgeNumber += 1
               
    
            else:
                for j in range(0, len(discreteTimeWindows[i])):
                        if(reaching_time <= discreteTimeWindows[i][j]):
                            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
                            edges['x[%i]' % edgeNumber] = (0, nodeCount[i][j])
                            edgeFlows[edgeNumber] = (0, nodeCount[i][j])
                            e.append(edgeNumber)
                            edgeNumber += 1
                            break
    edgesList[0] = e
   
    for i in range(0, len(nodeCount)):
        for j in range(0, len(nodeCount[i])-1):
            e = []
            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
            edges['x[%i]' % edgeNumber] = (nodeCount[i][j], nodeCount[i][j+1])
            edgeFlows[edgeNumber] = (nodeCount[i][j], nodeCount[i][j+1])
            e.append(edgeNumber)
            edgesList.append(e)
            edgeNumber += 1
        edgesList.append([])



    for i in range(len(discreteTimeWindows)):
        tu = nodeCount[i][-1]
        
        start_utime = discreteTimeWindows[i][0]
        end_utime = discreteTimeWindows[i][-1]

        # stores edges reachable from u
        for k in range(0, len(discreteTimeWindows)):
            sv = nodeCount[k][0]

            start_vtime = discreteTimeWindows[k][0]
            end_vtime = discreteTimeWindows[k][-1]

             # not reachable
            if(i != k and start_utime + timeMatrix[i][k] + serviceTime  > end_vtime):
                print("",end="")
                 

            elif(i != k and start_vtime  > end_utime + timeMatrix[i][k]+serviceTime):
                e = []
                x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))

                edges['x[%i]' % edgeNumber] = (tu, sv)  # u -> v reachable
                edgeFlows[edgeNumber] = (tu, sv)
#                 print(tu, "->", sv)
                e.append(edgeNumber)
                edgesList[tu].extend(e)
                edgeNumber += 1


            elif(i != k and start_vtime  == end_utime + timeMatrix[i][k]+serviceTime):
#                 print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)
                e = []
    #             x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
                x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))


                edges['x[%i]' % edgeNumber] = (tu, sv)  # u -> v reachable
#                 print(tu, "->", sv)
                edgeFlows[edgeNumber] = (tu, sv)
                e.append(edgeNumber)
                edgesList[tu].extend(e)
                edgeNumber += 1


            elif (i != k and start_vtime  < end_utime + timeMatrix[i][k]+serviceTime):

                firstReachableNode_index = 0  # first such that node >= sv
                for j in range(0, len(discreteTimeWindows[i])):
                    if(timeMatrix[i][k] + discreteTimeWindows[i][j]+serviceTime >= start_vtime):
                        firstReachableNode_index = j
                        break
                if(firstReachableNode_index > 0):
                    firstReachableNode_index=firstReachableNode_index-1
                # now starting from firstReachableNode make edges until feasible
                for l in range(firstReachableNode_index, len(discreteTimeWindows[i])):
                    e = []
                    reachingTime = discreteTimeWindows[i][l]+timeMatrix[i][k]+serviceTime
                    for r in range(0, len(discreteTimeWindows[k])):
                        if(discreteTimeWindows[k][r] >= reachingTime):
                            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
                            edges['x[%i]' % edgeNumber] =(nodeCount[i][l], nodeCount[k][r])  # u -> v reachable
                            edgeFlows[edgeNumber] = (nodeCount[i][l], nodeCount[k][r])
                            e.append(edgeNumber)
                            edgesList[nodeCount[i][l]].extend(e)
                            edgeNumber += 1
                            break




#     print('edge list b4 s t', edgesList)
#     print()

    # into t (except s)
    e = []
    for i in range(0, len(nodeCount)):
    #     x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
        edges['x[%i]' % edgeNumber] = (nodeCount[i][-1], -1)
        edgeFlows[edgeNumber] = (nodeCount[i][-1], -1)
        # add edge number in edge list
        edgesList[nodeCount[i][-1]].append(edgeNumber)
        edgeNumber += 1
    edgesList.append([])



    # GRAPH Construction OVER
    # if I add it to  0 edge i'll get flow for 1 at the same edge  , add to 1 customer 2 customer


    offset = edgeNumber

#     print('edge list', edgesList)
#     print()
#     print("edges", edges)
#     print()
#     print('edge number', edgeNumber)
#     print()

    # print("edge flows:", edgeFlows)

    flows = [0]*customers
    for i in range(customers):
        flows[i] = edgeFlows.copy()
        keys = edgeFlows.keys()
        for j in keys:  # stores key values of edgeflows i.e 0 1 2
            # all x[] > offset are for flow variables
    #         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))

            # remove key and replace it by edge
            t = flows[i].pop(j)
            flows[i][edgeNumber] = t
            edgeNumber += 1


    noc = 0
    # # contraint for outgoing flow for a customer
    for i in range(0, len(nodeCount)):  # choose one list from nodeCount
    #     contraint = [0] * (edgeNumber)
        coeff = [] # store where coeff = 1
        for j in range(0, len(nodeCount[i])):  # se
            node = nodeCount[i][j]
            for k in range(0, len(edgesList[node])):
                e = edgesList[node][k]
                u = edges['x[%i]' % e][0]
                v = edges['x[%i]' % e][1]
                    # print("u", u, "v", v)
                if(v not in nodeCount[i]):
                        # print('hello')
                        # print("u", u, "v", v)
                        coeff.append(x[e])

        m1.addConstr(( sum(coeff) == 1  ))
        noc+=1

#     print("outgoing edge lisst :")

    incomingEdgeList = []
    for i in range(0, len(edgesList)):
        incomingEdgeList.append([])  # adding lists in a list
    for i in edges:
        v = edges[i][1]  # u -> v

        edge_no = int(i[2:-1])
        incomingEdgeList[v].append(edge_no)

#     print("incoming edge list: ", incomingEdgeList)
#     print()

    #  constraint for incoming for a customer
    for i in range(0, len(nodeCount)):
        coeff = []
        contraint = [0] * (edgeNumber)  # indicates number of edges
        for j in range(0, len(nodeCount[i])):
            node = nodeCount[i][j]
            for k in range(0, len(incomingEdgeList[node])):
                e = incomingEdgeList[node][k]
                u = edges['x[%i]' % e][0]
                v = edges['x[%i]' % e][1]
                    # print("u", u, "v", v)
                if(u not in nodeCount[i]):
                        # print('hello')
                        # print("u", u, "v", v)
                        coeff.append(x[e])
        m1.addConstr(( sum(coeff) == 1  ))
        noc+=1

    # constraint for a node incoming = outgoing
    for i in range(0, len(nodeCount)):
        for j in range(0, len(nodeCount[i])):
            coeff = []
            contraint = [0] * (edgeNumber)
            node = nodeCount[i][j]
            for k in range(0, len(incomingEdgeList[node])):
                e = incomingEdgeList[node][k]
                contraint[e] = 1
                coeff.append(x[e])
            for k in range(0, len(edgesList[node])):
                e = edgesList[node][k]
                contraint[e] = -1
                coeff.append(-x[e])
            m1.addConstr(( sum(coeff) == 0 ))
            noc+=1  
            

               
    # #  0<= xij - fij    f<x => 0<=x-f
    for j in range(len(flows)):
        diff = offset*j+offset
        for k in flows[j].keys():
            coeff = [] 
            coeff.append(-x[k])
            coeff.append(x[k-diff])
            m1.addConstr(( sum(coeff) >= 0 ))
            

    # constraint for a particular customer outgoing == 0 for c the customer for rest dont care
    for c in range(customers):
        addn = offset*(c+1)
        for i in range(0, len(nodeCount)):
            
            if(i == c):
                
            # indicates number of edges
                coeff = []
                for j in range(0, len(nodeCount[i])):
                    
                    node = nodeCount[i][j]
                    for k in range(0, len(edgesList[node])):
                        
                        e = edgesList[node][k]
                        u = edges['x[%i]' % e][0]
                        v = edges['x[%i]' % e][1]
                        # print("u", u, "v", v)
                        if(v not in nodeCount[i]):
                            
                            # print('hello')
                            # print("u", u, "v", v)
                            edge = 'x[%i]' % e
                            node = edges[edge][0]
        #                     print("node:",node)
                            coeff.append(x[e+addn])
                m1.addConstr(( sum(coeff) == 0 ))


#     print('done w edges')        






#     print("CONSTRAINT for incoming = 1 ")
    for c in range(customers):
        
        addn = offset*(c+1)

        for i in range(0, len(nodeCount)):
            
            # indicates number of edges
            coeff = []
            if( i == c):
                for j in range(0, len(nodeCount[i])):
                    
                    node = nodeCount[i][j]
                    for k in range(0, len(incomingEdgeList[node])):
                        
                        e = incomingEdgeList[node][k]
                        u = edges['x[%i]' % e][0]
                        v = edges['x[%i]' % e][1]
                        # print("u", u, "v", v)
                        if(u not in nodeCount[i]):
                            
                            # print('hello')
                            # print("u", u, "v", v)
                            coeff.append(x[e+addn])
                m1.addConstr(( sum(coeff) == 1 ))




    # # flow conservation  for every node except those present in customer = c

    for c in range(customers):
        addn = offset*c+offset
        for i in range(0, len(nodeCount)):
            if ( i != c):
                for j in range(0, len(nodeCount[i])):
                    coeff = []
                    node = nodeCount[i][j]
                    for k in range(0, len(incomingEdgeList[node])):
                        e = incomingEdgeList[node][k]
                        coeff.append(x[e+addn])
                    for k in range(0, len(edgesList[node])):
                        e = edgesList[node][k]
                        coeff.append(-x[e+addn])
                    m1.addConstr(( sum(coeff) == 0 ))
                                                       
    
                                 
                                 
    m1.params.Method = 1
#     m1.params.Crossover = 0                           
    objectivefun = []  
    for i in range(0, len(edgesList[0])):
        objectivefun.append(x[edgesList[0][i]])
    obj = sum(objectivefun)
    m1.setObjective(obj,GRB.MINIMIZE)
 

   
    m1.optimize()
    total_runtime += m1.Runtime
    print("Runtime is :",m1.Runtime)
    print("Objective value : ",m1.objVal)
     
    # ans stores {1:0.5,4:0.9,...}   
    ans = {}
    for v in m1.getVars()[:offset]:
        if (v.x > 0):
            ans[(v.varName[2:-1])]=v.x

                                 
    new_edges = {}   # stores only those edges which are required   {e:(u,v)}
    for key in edges:
        if((key[2:-1]) in ans):
            new_edges[(key[2:-1])] = edges[key]
    
#     print(ans)
                                 
                                 
    graphForpaths = [] # will store outgoing edges
    for i in range(len(edgesList)): # edgesList contains outgoing from each node
        l = []
        for key in new_edges:
            if(new_edges[key][0] == i):  # u = i
                if(new_edges[key][1] == -1):
                    l.append(len(edgesList)-1)
                else:
                    l.append(new_edges[key][1])
        graphForpaths.append(l)
        
    allcustomer = [] # will store customers from longest path + ones path for del 
    # --------------------------------------------------- 
    # ONES PATH
    edges1=[] # stores outgoing edges wtih val > 0
    for i in range(nodeCount[-1][-1]+1):
        edges1.append([])
    for key in new_edges.keys():
        edges1[new_edges[key][0]].append(key)
        
    paths = onesPath(graphForpaths,edges1)  # paths contains all ones path
    for i in range(len(paths)):
        paths[i][-1] = -1
#     print("Ones paths",paths)
   
    for route in paths:
        distict_customers,customer_nodes = distinctCustomer(route,nodeCount)
        customer_occurence = customer_nodes.copy()
        allcustomer+=customer_nodes
        customers_covered += distict_customers
        
        #----------------------------
        final_str = str()
        final_str = final_str + 'Ones path : Source -> '
    
       
        
        for i in customer_occurence:
            customer_number = searchKey(i,cust_dict)  # search key with value = i (i represents customer)
            final_str = final_str + ''.join(str(customer_number)+ ' -> ')


        final_str = final_str+' Sink '

        final_path_list.append(final_str)

        customer_nodes.sort()  # will store customers in ascending order
        for i in customer_nodes:
            for key, value in cust_dict.items():
                if i == value:
                    cust_dict[key]= -(VEHICLES+1)
        
        VEHICLES+=1
        #---------------------------------------------------------    
    #----------------------------------------------------
    
    
#     print("Edges",new_edges)
#     print("ANS",ans)
    visited = [False]*len(graphForpaths)
    ts = topologicalSort(visited,graphForpaths)
    
    while(True):
        # dp[i] stores maximum length from s to i
        path = [0]*len(ts)                     # stores index of path for which max occurs at i
        dp = [0]*len(ts)
        dp[0] = 1
        for i in range(1,len(ts)):
            u = ts[i]
            for j in range(0,i):
                v = ts[j]
                if(u in graphForpaths[v]): # egde to a prev node always exists
                    # v -> u and if path from s to v contains a node x st cust(x) = cust(u) then also add 0
                    val = length(u,v,nodeCount,path,j,ts)
                    if(val + dp[j] > dp[i]):
                        dp[i] = val + dp[j]
                        path[i] = j
    #             dp[i] = max(dp[i],length(u,v,nodeCount) + dp[j])
    #     print("max length :",dp[-1]-2)
    #     print(path)
    
#         print("index path:",path)
        route = []
        route.append(-1)
        curr = -1
        while(True):
            if(curr == 0):
                break
            prev = path[curr]
            route.append(ts[prev])
            curr = prev

        for l in range(1,len(route)-1):
            del ts[ts.index(route[l])]
            
            
        route.reverse()
        print("other routes :",route)
        if route not in paths:
            print("longest route (repeat):",route)
            break

    
    # now i need longest 
    distict_customers,customer_nodes = distinctCustomer(route,nodeCount)
    customer_occurence = customer_nodes.copy() # customer node stores index of visited customer in order of their occ in routes
    customers_covered+=distict_customers 
#     print("customer_occurence  -->",customer_occurence)
    
#     if(len(customer_nodes) == 0 and customers_covered == customers): # i.e no path other than one exist
#         print("breaking.......")
#         break
    if(len(customer_nodes) == 0): # i.e no path other than one exist
        print("Total customers covered",customers_covered)
        break
    
    
    
    # new addition-----------------
    final_str = str()
    final_str = final_str + 'Source -> '
    
    
#     print("customer dixt->",cust_dict)
    for i in customer_occurence:
        customer_number = searchKey(i,cust_dict)     # search key with value = i (i represents customer)
        final_str = final_str + ''.join(str(customer_number)+ ' -> ')
        
        
    final_str = final_str+' Sink '

    final_path_list.append(final_str)
    
    customer_nodes.sort()  # will store customers in ascending order
    for i in customer_nodes:
        for key, value in cust_dict.items():
            if i == value:
                cust_dict[key]= -(VEHICLES+1)
                
    a = 0
    for i in range(len(cust_dict)):
        if cust_dict[i]>= 0 :
            cust_dict[i] = a
            a+=1
    #----------------------------
    
    
    allcustomer += customer_nodes
    allcustomer = np.unique(allcustomer)
    allcustomer = allcustomer[::-1]
#     customers_covered += len(allcustomer)
    
#     print("customer covered",customers_covered)
    #modify time-windows
#     print("customer nodes",allcustomer)
    for i in range(0,len(allcustomer)):
        del timeWindows[allcustomer[i]]
        
    #-------------------------
    for i in range(0,len(allcustomer)):
        del time_from_source[allcustomer[i]]
    #----------------------------------
    
                                 
    discreteTimeWindows = []
    for i in timeWindows:
        discreteTimeWindows.append(discreteTime(i, 10))
                                 

    for row in allcustomer:
        del timeMatrix[row]

    for row in range(0,len(timeMatrix)):                         
        for col in allcustomer:
            del timeMatrix[row][col]
    
    print("                                                                                      ")
    print("Removing ",distict_customers," distinct customer",customer_nodes,"and path is ",route)
    print("----------------------------------------------------------------------------------------------------")
    print("                                                                                      ")
    
#     if(distict_customers != 0):
#         VEHICLES+=1 
    VEHICLES+=1 

In [ ]:
print("Vehicles required : ",VEHICLES)
print("Total Runtime",total_runtime)
print("Number of Iterations",Iterations)

In [ ]:
print(file)
final_path_list

In [ ]:
# C101.csv
# ['Source -> 19 -> 23 -> 24 -> 18 -> 14 -> 15 -> 5 -> 22 -> 21 -> 20 ->  Sink ',
#  'Source -> 12 -> 16 -> 17 -> 9 -> 10 -> 13 -> 3 -> 1 -> 0 ->  Sink ',
#  'Ones path : Source -> 4 -> 2 -> 6 -> 7 -> 8 -> 11 ->  Sink ']

# C102.csv
# ['Source -> 19 -> 23 -> 24 -> 18 -> 14 -> 15 -> 5 -> 0 -> 21 -> 11 -> 1 ->  Sink ',
#  'Source -> 12 -> 16 -> 4 -> 9 -> 6 -> 13 -> 3 -> 2 -> 20 ->  Sink ',
#  'Ones path : Source -> 17 -> 7 -> 10 -> 8 -> 22 ->  Sink ']

# C104.csv
# ['Source -> 19 -> 23 -> 24 -> 20 -> 21 -> 6 -> 1 -> 9 -> 4 -> 5 -> 0 -> 2 ->  Sink ',
#  'Source -> 17 -> 18 -> 13 -> 15 -> 11 -> 8 -> 16 -> 3 ->  Sink ',
#  'Source -> 12 -> 7 -> 14 -> 10 -> 22 ->  Sink ']

# C105.csv
# ['Source -> 19 -> 23 -> 17 -> 9 -> 15 -> 13 -> 11 -> 22 -> 21 -> 20 ->  Sink ',
#  'Ones path : Source -> 12 -> 16 -> 24 -> 18 -> 10 -> 0 ->  Sink ',
#  'Ones path : Source -> 4 -> 2 -> 6 -> 7 -> 14 -> 8 -> 5 -> 3 -> 1 ->  Sink ']

# C106.csv
# ['Source -> 19 -> 23 -> 24 -> 18 -> 14 -> 15 -> 8 -> 22 -> 21 -> 20 ->  Sink ',
#  'Ones path : Source -> 12 -> 16 -> 17 -> 9 -> 10 -> 5 -> 3 ->  Sink ',
#  'Ones path : Source -> 4 -> 2 -> 6 -> 7 -> 13 -> 11 -> 1 -> 0 ->  Sink ']

# C107.csv
# ['Source -> 19 -> 2 -> 6 -> 7 -> 9 -> 10 -> 8 -> 5 -> 22 -> 21 -> 20 ->  Sink ',
#  'Source -> 23 -> 16 -> 17 -> 18 -> 14 -> 15 -> 11 -> 3 -> 1 -> 0 ->  Sink ',
#  'Ones path : Source -> 4 -> 12 -> 24 -> 13 ->  Sink ']

# C108.csv
# ['Source -> 19 -> 24 -> 23 -> 9 -> 7 -> 10 -> 8 -> 5 -> 3 -> 0 -> 20 ->  Sink ',
#  'Source -> 4 -> 6 -> 12 -> 17 -> 18 -> 15 -> 13 -> 11 -> 1 -> 21 ->  Sink ',
#  'Ones path : Source -> 16 -> 2 -> 14 -> 22 ->  Sink ']

# C109.csv
# ['Source -> 19 -> 2 -> 4 -> 6 -> 9 -> 10 -> 8 -> 1 -> 3 -> 22 -> 21 ->  Sink ',
#  'Source -> 24 -> 23 -> 16 -> 17 -> 18 -> 14 -> 15 -> 13 -> 11 -> 0 -> 20 ->  Sink ',
#  'Ones path : Source -> 7 -> 12 -> 5 ->  Sink ']

In [ ]:
RC101.csv
['Source -> 13 -> 14 -> 10 -> 15 -> 9 -> 12 -> 16 ->  Sink ',
 'Ones path : Source -> 4 -> 6 -> 5 -> 7 -> 2 -> 0 -> 3 ->  Sink ',
 'Source -> 22 -> 20 -> 18 -> 21 -> 23 -> 24 ->  Sink ',
 'Ones path : Source -> 17 -> 19 ->  Sink ',
 'Ones path : Source -> 1 -> 11 -> 8 ->  Sink ']

RC102.csv
['Source -> 13 -> 14 -> 15 -> 8 -> 9 -> 12 -> 16 -> 11 ->  Sink ',
 'Source -> 2 -> 1 -> 6 -> 5 -> 7 -> 4 -> 0 -> 3 ->  Sink ',
 'Ones path : Source -> 18 -> 22 -> 17 -> 24 ->  Sink ',
 'Ones path : Source -> 10 -> 21 -> 19 -> 23 -> 20 ->  Sink ']

RC103.csv
['Ones path : Source -> 19 -> 18 -> 22 -> 23 -> 21 -> 20 -> 24 -> 17 ->  Sink ',
 'Source -> 15 -> 10 -> 14 -> 8 -> 9 -> 12 -> 16 -> 13 ->  Sink ',
 'Source -> 2 -> 4 -> 0 -> 5 -> 6 -> 3 -> 1 ->  Sink ',
 'Ones path : Source -> 7 -> 11 ->  Sink ']

RC104.csv
['Source -> 21 -> 20 -> 22 -> 23 -> 18 -> 19 -> 24 -> 17 ->  Sink ',
 'Source -> 13 -> 16 -> 10 -> 8 -> 9 -> 11 -> 15 -> 12 ->  Sink ',
 'Source -> 0 -> 4 -> 2 -> 1 -> 7 -> 6 -> 3 -> 5 ->  Sink ',
 'Ones path : Source -> 14 ->  Sink ']

RC105.csv
['Source -> 20 -> 22 -> 18 -> 21 -> 23 -> 19 -> 24 ->  Sink ',
 'Source -> 6 -> 2 -> 4 -> 7 -> 5 -> 1 -> 3 ->  Sink ',
 'Source -> 14 -> 15 -> 8 -> 0 ->  Sink ',
 'Source -> 10 -> 13 -> 11 -> 9 -> 16 ->  Sink ',
 'Ones path : Source -> 17 -> 12 ->  Sink ']

RC106.csv
['Source -> 13 -> 14 -> 11 -> 15 -> 9 -> 8 -> 12 -> 16 ->  Sink ',
 'Source -> 22 -> 20 -> 17 -> 21 -> 19 -> 23 -> 24 ->  Sink ',
 'Source -> 4 -> 7 -> 1 -> 6 -> 5 -> 0 -> 3 ->  Sink ',
 'Source -> 10 -> 18 ->  Sink ',
 'Ones path : Source -> 2 ->  Sink ']

RC107.csv
['Ones path : Source -> 4 -> 6 -> 1 -> 2 -> 7 -> 5 -> 3 -> 0 ->  Sink ',
 'Source -> 10 -> 11 -> 13 -> 16 -> 15 -> 14 -> 9 -> 12 ->  Sink ',
 'Source -> 22 -> 20 -> 17 -> 18 -> 19 -> 23 -> 24 ->  Sink ',
 'Ones path : Source -> 21 -> 8 ->  Sink ']

RC108.csv
['Ones path : Source -> 22 -> 20 -> 24 -> 17 -> 18 -> 21 -> 19 -> 23 ->  Sink ',
 'Source -> 1 -> 6 -> 7 -> 5 -> 3 -> 0 -> 2 -> 4 ->  Sink ',
 'Source -> 14 -> 15 -> 11 -> 10 -> 8 -> 9 -> 12 -> 13 ->  Sink ',
 'Ones path : Source -> 16 ->  Sink ']

In [ ]:
# RC101.csv
# ['Ones path : Source -> 32 -> 27 -> 26 -> 25 -> 33 ->  Sink ',
#  'Ones path : Source -> 28 -> 30 -> 29 -> 49 -> 31 ->  Sink ',
#  'Source -> 44 -> 1 -> 6 -> 5 -> 45 -> 2 -> 3 -> 0 ->  Sink ',
#  'Source -> 38 -> 41 -> 43 -> 37 -> 39 -> 36 -> 34 ->  Sink ',
#  'Ones path : Source -> 4 -> 40 -> 42 ->  Sink ',
#  'Source -> 22 -> 17 -> 18 -> 21 -> 19 -> 47 -> 24 ->  Sink ',
#  'Ones path : Source -> 20 -> 48 -> 12 ->  Sink ',
#  'Source -> 13 -> 46 -> 10 -> 15 -> 8 -> 9 -> 16 ->  Sink ',
#  'Ones path : Source -> 35 -> 7 ->  Sink ',
#  'Ones path : Source -> 14 -> 11 -> 23 ->  Sink ']

# RC102.csv
# ['Source -> 2 -> 44 -> 4 -> 7 -> 5 -> 45 -> 6 -> 3 -> 1 ->  Sink ',
#  'Source -> 33 -> 32 -> 27 -> 29 -> 30 -> 25 -> 31 -> 49 ->  Sink ',
#  'Source -> 46 -> 14 -> 10 -> 15 -> 9 -> 12 -> 16 -> 11 ->  Sink ',
#  'Source -> 41 -> 38 -> 43 -> 37 -> 39 -> 42 -> 34 ->  Sink ',
#  'Ones path : Source -> 26 -> 28 -> 19 ->  Sink ',
#  'Ones path : Source -> 13 -> 8 -> 0 ->  Sink ',
#  'Source -> 47 -> 22 -> 18 -> 21 -> 48 -> 20 -> 24 ->  Sink ',
#  'Ones path : Source -> 36 -> 35 -> 40 -> 23 ->  Sink ',
#  'Ones path : Source -> 17 ->  Sink ']

# RC103.csv
# ['Source -> 19 -> 47 -> 17 -> 22 -> 20 -> 48 -> 24 -> 18 -> 23 ->  Sink ',
#  'Source -> 13 -> 10 -> 14 -> 8 -> 9 -> 46 -> 15 -> 16 -> 11 ->  Sink ',
#  'Source -> 6 -> 44 -> 45 -> 5 -> 7 -> 4 -> 2 -> 3 -> 1 ->  Sink ',
#  'Source -> 41 -> 35 -> 39 -> 37 -> 40 -> 36 -> 34 -> 42 ->  Sink ',
#  'Source -> 33 -> 32 -> 31 -> 29 -> 30 -> 27 -> 49 ->  Sink ',
#  'Source -> 28 -> 25 -> 26 -> 0 ->  Sink ',
#  'Ones path : Source -> 38 -> 43 -> 12 ->  Sink ',
#  'Ones path : Source -> 21 ->  Sink ']

# RC104.csv
# ['Source -> 33 -> 28 -> 26 -> 25 -> 29 -> 32 -> 31 -> 49 ->  Sink ',
#  'Source -> 23 -> 21 -> 20 -> 22 -> 17 -> 47 -> 18 -> 24 -> 48 ->  Sink ',
#  'Source -> 41 -> 40 -> 43 -> 37 -> 42 -> 39 -> 34 -> 35 -> 38 ->  Sink ',
#  'Source -> 46 -> 16 -> 10 -> 8 -> 9 -> 15 -> 12 -> 13 -> 11 ->  Sink ',
#  'Source -> 2 -> 0 -> 45 -> 7 -> 6 -> 1 -> 3 -> 4 -> 44 ->  Sink ',
#  'Source -> 19 -> 27 -> 30 ->  Sink ',
#  'Ones path : Source -> 14 -> 5 -> 36 ->  Sink ']

# RC105.csv
# ['Source -> 6 -> 1 -> 44 -> 4 -> 7 -> 45 -> 3 -> 2 -> 0 ->  Sink ',
#  'Ones path : Source -> 13 -> 14 -> 46 -> 8 -> 9 -> 12 ->  Sink ',
#  'Ones path : Source -> 11 -> 15 -> 10 -> 5 -> 16 ->  Sink ',
#  'Source -> 41 -> 38 -> 35 -> 37 -> 39 -> 42 -> 34 -> 36 ->  Sink ',
#  'Source -> 27 -> 32 -> 29 -> 28 -> 25 -> 33 -> 49 ->  Sink ',
#  'Ones path : Source -> 43 -> 40 -> 30 -> 31 ->  Sink ',
#  'Ones path : Source -> 26 -> 17 -> 20 -> 47 -> 23 ->  Sink ',
#  'Ones path : Source -> 18 -> 22 -> 21 -> 48 -> 19 -> 24 ->  Sink ']

# RC106.csv
# ['Source -> 41 -> 43 -> 40 -> 38 -> 39 -> 42 -> 36 -> 34 ->  Sink ',
#  'Source -> 22 -> 18 -> 21 -> 20 -> 17 -> 48 -> 47 -> 24 ->  Sink ',
#  'Source -> 13 -> 46 -> 11 -> 14 -> 8 -> 12 -> 9 -> 16 ->  Sink ',
#  'Source -> 44 -> 4 -> 7 -> 5 -> 6 -> 45 -> 2 -> 3 ->  Sink ',
#  'Ones path : Source -> 10 -> 15 -> 19 -> 23 ->  Sink ',
#  'Ones path : Source -> 1 -> 35 -> 37 -> 0 ->  Sink ',
#  'Source -> 32 -> 27 -> 28 -> 29 -> 33 -> 49 ->  Sink ',
#  'Ones path : Source -> 26 -> 30 -> 31 -> 25 ->  Sink ']

# RC107.csv
# ['Source -> 13 -> 16 -> 46 -> 11 -> 15 -> 10 -> 8 -> 9 -> 12 ->  Sink ',
#  'Source -> 1 -> 5 -> 6 -> 7 -> 4 -> 2 -> 3 -> 44 -> 0 ->  Sink ',
#  'Source -> 41 -> 39 -> 43 -> 40 -> 36 -> 37 -> 42 -> 35 -> 38 ->  Sink ',
#  'Ones path : Source -> 14 -> 45 -> 34 ->  Sink ',
#  'Source -> 22 -> 24 -> 20 -> 48 -> 23 -> 21 -> 19 -> 47 ->  Sink ',
#  'Source -> 28 -> 30 -> 27 -> 32 -> 33 -> 29 -> 31 ->  Sink ',
#  'Source -> 18 -> 17 -> 25 -> 26 ->  Sink ',
#  'Ones path : Source -> 49 ->  Sink ']

# RC108.csv
# ['Source -> 43 -> 40 -> 41 -> 39 -> 42 -> 37 -> 34 -> 35 -> 38 ->  Sink ',
#  'Source -> 22 -> 18 -> 47 -> 48 -> 17 -> 24 -> 19 -> 23 ->  Sink ',
#  'Ones path : Source -> 20 -> 21 -> 49 -> 36 ->  Sink ',
#  'Source -> 11 -> 16 -> 14 -> 12 -> 15 -> 9 -> 46 -> 13 ->  Sink ',
#  'Source -> 44 -> 1 -> 45 -> 5 -> 7 -> 4 -> 2 -> 0 ->  Sink ',
#  'Ones path : Source -> 6 -> 3 -> 8 -> 10 ->  Sink ',
#  'Source -> 30 -> 31 -> 27 -> 29 -> 26 -> 33 -> 32 ->  Sink ',
#  'Ones path : Source -> 25 -> 28 ->  Sink ']

##### 